In [5]:
import os
import cv2
import numpy as np
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.datasets import fetch_lfw_people
from sklearn.neighbors import KNeighborsClassifier
import random
from tensorflow.keras.models import load_model

# 1. Load and preprocess FER2013 dataset from your custom path
def load_emotion_dataset(data_dir=r'C:\Users\raaga\OneDrive\Desktop\fer\train', target_size=(48, 48)):
    images = []
    labels = []
    label_names = sorted(os.listdir(data_dir))
    
    for label in label_names:
        label_dir = os.path.join(data_dir, label)
        for file in os.listdir(label_dir):
            img_path = os.path.join(label_dir, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, target_size)
                images.append(img)
                labels.append(label)

    images = np.array(images, dtype="float32") / 255.0
    images = np.expand_dims(images, -1)
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    return images, labels, lb.classes_

X_emotion, y_emotion, emotion_labels = load_emotion_dataset()

# 2. Train Emotion Classification Model
X_train_em, X_val_em, y_train_em, y_val_em = train_test_split(X_emotion, y_emotion, test_size=0.2, random_state=42)

model_emotion = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(emotion_labels), activation='softmax')
])

model_emotion.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(patience=3, restore_best_weights=True)
model_emotion.fit(X_train_em, y_train_em, validation_data=(X_val_em, y_val_em), epochs=15, batch_size=64, callbacks=[early_stop])

model_emotion.save("emotion_model.h5")

# 3. Load LFW dataset (3–5 students)
lfw = fetch_lfw_people(min_faces_per_person=20, resize=0.4)
student_names = list(np.unique(lfw.target_names))
chosen_students = random.sample(student_names, 4)
X_faces = []
y_faces = []

for i, name in enumerate(lfw.target_names):
    if name in chosen_students:
        X_faces.append(lfw.images[i])
        y_faces.append(name)

X_faces = np.array(X_faces)
y_faces = np.array(y_faces)

# 4. Train simple Face Recognition model
X_faces_flatten = X_faces.reshape((X_faces.shape[0], -1))
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_faces_flatten, y_faces)

# 5. Simulate attendance within 9:30–10:00 AM
def simulate_attendance():
    attendance = []
    now = datetime.datetime.now()
    current_time = now.time()
    start_time = datetime.time(9, 30)
    end_time = datetime.time(10, 0)

    if not (start_time <= current_time <= end_time):
        print("❌ System is active only between 9:30 AM to 10:00 AM.")
        return

    for i in range(len(X_faces)):
        face = X_faces[i]
        name = y_faces[i]
        face_flat = face.flatten().reshape(1, -1)

        predicted_name = knn.predict(face_flat)[0]

        # Predict emotion using trained model
        face_resized = cv2.resize(face, (48, 48))
        face_norm = face_resized.astype("float32") / 255.0
        face_input = np.expand_dims(face_norm, axis=(0, -1))
        emotion_pred = model_emotion.predict(face_input)
        emotion = emotion_labels[np.argmax(emotion_pred)]

        attendance.append({
            'Name': predicted_name,
            'Time': now.strftime("%H:%M:%S"),
            'Status': 'Present',
            'Emotion': emotion
        })

    df = pd.DataFrame(attendance)
    df.to_csv("attendance.csv", index=False)
    print("✅ Attendance saved to attendance.csv")

simulate_attendance()


Epoch 1/15
197/197 ━━━━━━━━━━━━━━━━━━━━ 18s 75ms/step - accuracy: 0.4597 - loss: 1.1557 - val_accuracy: 0.5284 - val_loss: 1.0475
Epoch 2/15
197/197 ━━━━━━━━━━━━━━━━━━━━ 13s 65ms/step - accuracy: 0.5676 - loss: 0.9852 - val_accuracy: 0.6075 - val_loss: 0.9202
Epoch 3/15
197/197 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - accuracy: 0.6291 - loss: 0.8739 - val_accuracy: 0.6173 - val_loss: 0.8788
Epoch 4/15
197/197 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - accuracy: 0.6471 - loss: 0.8212 - val_accuracy: 0.6418 - val_loss: 0.8493
Epoch 5/15
197/197 ━━━━━━━━━━━━━━━━━━━━ 13s 67ms/step - accuracy: 0.6765 - loss: 0.7690 - val_accuracy: 0.6405 - val_loss: 0.8258
Epoch 6/15
197/197 ━━━━━━━━━━━━━━━━━━━━ 13s 65ms/step - accuracy: 0.6955 - loss: 0.7260 - val_accuracy: 0.6545 - val_loss: 0.8171
Epoch 7/15
197/197 ━━━━━━━━━━━━━━━━━━━━ 13s 64ms/step - accuracy: 0.7107 - loss: 0.6857 - val_accuracy: 0.6643 - val_loss: 0.8010
Epoch 8/15
197/197 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - accuracy: 0.7336 - loss: 0.6432 - 

❌ System is active only between 9:30 AM to 10:00 AM.
